In [3]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]
X = tf.placeholder('float', [None, 4])
Y = tf.placeholder('float', [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activation
# softmax = exp(logitx)/reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# cross entropy   cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis)))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X: x_data, Y: y_data})
        
        if step % 200 == 0:
            print(step, cost_val)
            
    
    print('-----------')
    # Testing & one-hot encoding
    a = sess.run(hypothesis, feed_dict={X:[[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a,1))) 
    
    print('-----------')
    b = sess.run(hypothesis, feed_dict = {X:[[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))
    
    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))


Instructions for updating:
Colocations handled automatically by placer.
0 113.19617
200 62.783966
400 47.24459
600 49.87044
800 0.3098936
1000 0.029472362
1200 0.016726311
1400 0.012681568
1600 0.010575234
1800 0.009266796
2000 0.008384263
-----------
[[8.969651e-13 1.000000e+00 5.833288e-14]] [1]
-----------
[[9.9994278e-01 5.7187262e-05 2.5131312e-08]] [0]
--------------
[[1.3681111e-37 5.6954543e-14 1.0000000e+00]] [2]
--------------
[[8.9696512e-13 1.0000000e+00 5.8332882e-14]
 [9.9994278e-01 5.7187262e-05 2.5131312e-08]
 [1.3681111e-37 5.6954543e-14 1.0000000e+00]] [1 0 2]


### softmax_cross_entropy_with_logits()

In [4]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16), (101, 1)
'''

nb_classes = 7 # 0~6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0~6

Y_one_hot = tf.one_hot(Y, nb_classes)
print('one hot', Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print('reshaped Y_one_hot', Y_one_hot)

'''
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
'''

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
# cross entropy
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                       feed_dict={X: x_data, Y: y_data})
        
        if step % 500 == 0:
            print('step:{:5}\tCost: {:.3f}\tAcc:{:.2%}'.format(step, cost_val, acc_val))
    
    # check prediction
    pred = sess.run(prediction, feed_dict={X:x_data})
    # y_data = (N,1) = flatten ->  (N,) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print('[{}] Prediction: {} True, Y: {}'.format(p==int(y), p, int(y)))
        
'''
step:    0	Cost: 10.652	Acc:2.97%
step:  500	Cost: 0.232	Acc:93.07%
step: 1000	Cost: 0.118	Acc:97.03%
step: 1500	Cost: 0.076	Acc:100.00%
step: 2000	Cost: 0.056	Acc:100.00%
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 3 True, Y: 3
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 3 True, Y: 3
[True] Prediction: 3 True, Y: 3
'''

(101, 16) (101, 1)
one hot Tensor("one_hot_3:0", shape=(?, 1, 7), dtype=float32)
reshaped Y_one_hot Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)
step:    0	Cost: 10.652	Acc:2.97%
step:  500	Cost: 0.232	Acc:93.07%
step: 1000	Cost: 0.118	Acc:97.03%
step: 1500	Cost: 0.076	Acc:100.00%
step: 2000	Cost: 0.056	Acc:100.00%
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 3 True, Y: 3
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 3 True, Y: 3
[True] Prediction: 3 True, Y: 3
[True] Prediction: 0 True, Y: 0
[True] Prediction: 0 True, Y: 0
[True] Prediction: 1 True, Y: 1
[True] Prediction: 3 True, Y: 3
[True] Prediction: 6 True, Y: 6
[True] Prediction: 6 True, Y: 6
[True] Prediction: 6 True, Y: 6
[True] Prediction: 1 True, Y: 1
[True] Prediction: 0 True, Y: 0
[True] Prediction: 3 True, Y: 3
[True] Prediction: 0 True, Y: 0
[True] Prediction: 1 True, Y: 1
[True